In [1]:
from sklearn.svm import SVC
import functions
from sklearn.metrics import balanced_accuracy_score
# Paso 1: Cargar los datos

data_test= functions.load_data("../files/input/test_data.csv.zip")
data_train= functions.load_data("../files/input/train_data.csv.zip")

# Paso 2: Limpiar los datos
data_test = functions.clean_data(data_test)
data_train = functions.clean_data(data_train)
# paso 3: dividir los datos 
x_test=data_test.drop("default", axis=1)
y_test=data_test[["default"]]
x_train=data_train.drop("default", axis=1)
y_train=data_train[["default"]]
# Paso 4: Crear un pipeline para el modelo de clasificación
pipeline = functions.make_pipeline(
    estimator=SVC(random_state=42))
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['EDUCATION', 'SEX',
                                                   'MARRIAGE'])])),
                ('pca', PCA()), ('num', StandardScaler()),
                ('kbest', SelectKBest(k=1)),
                ('estimator', SVC(random_state=42))])

In [2]:
# Paso 5: Definir los hiperparámetros para la búsqueda en cuadrícula


param_grid = {
    'kbest__k': [4],
    'estimator__C': [13.5],
    'estimator__kernel': ['rbf'],
    'estimator__gamma': ['auto'],
    'estimator__degree': [2],

}

In [3]:
#Paso 6: Crear el objeto GridSearchCV

estimator = functions.make_grid_search(estimator=pipeline, param_grid=param_grid, cv=10)

# Paso 7: Ajustar el modelo a los datos de entrenamiento
estimator.fit(x_train, y_train)
print("Mejor modelo: ")
print(estimator)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


c:\SML_Works\Experimentos\Analitica_Predictiva\Laboratorios\Lab3\2024-2-LAB-03-prediccion-del-default-usando-svc-smurilloo\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\SML_Works\Experimentos\Analitica_Predictiva\Laboratorios\Lab3\2024-2-LAB-03-prediccion-del-default-usando-svc-smurilloo\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mejor modelo: 
GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['EDUCATION',
                                                                          'SEX',
                                                                          'MARRIAGE'])])),
                                       ('pca', PCA()),
                                       ('num', StandardScaler()),
                                       ('kbest', SelectKBest(k=1)),
                                       ('estimator', SVC(random_state=42))]),
             n_jobs=-1,
             param_grid={'estimator__C': [13.5], 'estimator__degree': [2],
        

In [4]:
# Paso 8: Obtener el mejor estimador
best_estimator = functions.load_estimator_compressed()

if best_estimator is not None:

    saved_balanced_accuracy = balanced_accuracy_score(
        y_true=y_test, y_pred=best_estimator.predict(x_test)
    )

    current_balanced_accuracy = balanced_accuracy_score(
        y_true=y_test, y_pred=estimator.predict(x_test)
    )

    if current_balanced_accuracy < saved_balanced_accuracy:
        estimator = best_estimator

functions.save_estimator_compressed(estimator)

# Ejecutar cálculo de métricas y matrices
functions.calculate_and_save_metrics(estimator, x_train, x_test, y_train, y_test)
functions.calculate_and_save_confusion_matrices(estimator, x_train, x_test, y_train, y_test)

# Paso 1: Obtener el mejor modelo de GridSearchCV
best_model = estimator.best_estimator_

# Paso 2: Hacer predicciones con el mejor modelo
y_train_pred = best_model.predict(x_train)
y_test_pred = best_model.predict(x_test)
# Imprimir métricas para el conjunto de entrenamiento
functions.print_metric(y_train, y_train_pred, 'Entrenamiento')
functions.print_metric(y_test, y_test_pred, 'prueba')

# Imprimir métricas para el conjunto de prueba
functions.print_confusion_matrix(y_test, y_test_pred, '')

Métricas para el dataset Entrenamiento:
Accuracy: 0.8209
Precision: 0.6921
Recall: 0.3706
F1 Score: 0.4827
----------------------------------------
Métricas para el dataset prueba:
Accuracy: 0.8297
Precision: 0.6737
Recall: 0.3835
F1 Score: 0.4888
----------------------------------------
Matriz de Confusión para el dataset :
[[6719  354]
 [1175  731]]
----------------------------------------
